In [2]:
from dotenv import load_dotenv
import os
import importlib
import sys 

sys.path.append('/home1/yongpeng/projects/dso599-llm-chatbot/llm_chatbot')

# Load environment variables from .env file
load_dotenv('../llm_chatbot/.env')

True

# Part 2 Vector Store

In [5]:
import vector_store.vector_store
importlib.reload(vector_store.vector_store)
from vector_store.vector_store import load_documents, split_documents, create_brand_new_chroma_db, create_retriver_tool_chroma_db, get_embedding_function

In [3]:
documents = load_documents('../pdf_data/')
chunks = split_documents(documents)
create_brand_new_chroma_db(chunks, db_path='../chroma_vector_store/')

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)


Chroma DB created at ../chroma_vector_store/ with 14 chunks.


In [6]:
# load from disk
from langchain.vectorstores.chroma import Chroma

db3 = Chroma(persist_directory='../chroma_vector_store/', embedding_function=get_embedding_function())
docs = db3.similarity_search('What is the safe temperature for T-Rex Transport?')
print(docs[0].page_content)

Recording Temperatures using Planet and Responding to Planet Alerts- Rev#10 Downloaded or Printed documents should be deleted/shredded within 1 week. Page 1 of 3  JP-ZOO-INST-24: Recording Temperatures for Transport of T-Rexs   1. PURPOSE This Work Instruction describes the DINO Transporter’s responsibility of keeping the T-Rexs safe if the current temperature is out of the safe temperature range, which is 57°F-68°F for T-Rexs. SCOPE This Work Instruction applies to DINO Operations Staff and teammates in the United States. 2. SAFETY There are no direct safety implications implicated by this procedure. 3. RESPONSIBILITY Transporters Record temperatures from the TYR55 sensor located in the cargo bay of every dinosaur freighter on the Research Dinosaur Welfare Check Log. T-Rexs are not safe when temperatures are out of range (57°F-68°F). Associate Director, Operations and Strategy Responsible for the action plan when temperatures in the freighter cannot be controlled and for Alert


### Create Agent

In [10]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI, ChatOpenAI
import openai

import react_agent
importlib.reload(react_agent)
from react_agent import get_react_agent, OpenAI_agent


import vector_store
importlib.reload(vector_store)
from vector_store.vector_store import load_documents, split_documents, create_brand_new_chroma_db, create_retriver_tool_chroma_db

In [7]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [11]:
tools = [create_retriver_tool_chroma_db(db_path='/home1/yongpeng/projects/dso599-llm-chatbot/chroma_vector_store/')]
agent_executor = OpenAI_agent(model="gpt-3.5-turbo",
                              tools=tools)


In [12]:
agent_executor.invoke({"input": "What is the safe temperature for T-Rex Transport?"})



> Entering new AgentExecutor chain...
I should use the dinosaur_temperature_transport_info tool to find the safe temperature for T-Rex transport.
Action: dinosaur_temperature_transport_info
Action Input: T-Rexlocation and the actual time the T-Rexs were moved and the time they were moved back into the freighter. 6.2.10 If the transporter is unreachable, all efforts must be made to keep reaching out to the transporter till the transporter responds to the alert.  7 QUALITY DOCUMENTS 7.1 Research Dinosaur Welfare Check Log       XXXXXX   8 REFERENCES N/A

Recording Temperatures using Planet and Responding to Planet Alerts- Rev#10 Downloaded or Printed documents should be deleted/shredded within 1 week. Page 1 of 3  JP-ZOO-INST-24: Recording Temperatures for Transport of T-Rexs   1. PURPOSE This Work Instruction describes the DINO Transporter’s responsibility of keeping the T-Rexs safe if the current temperature is out of the safe temperature range, which is 57°F-68°F for T-Rexs. SCOPE T

{'input': 'What is the safe temperature for T-Rex Transport?',
 'output': 'The safe temperature for T-Rex transport is between 57°F-68°F.'}

# Part 3

### DynamoDB Database

In [13]:
import aws_features.dynamo_db
importlib.reload(aws_features.dynamo_db)
from aws_features.dynamo_db import *

In [14]:
dynamodb_args={'table_name': 'dino_info',
                'csv_file_path': '../csv_data/dino_records.csv'}

In [16]:
if not check_table_exists(dynamodb_args['table_name']):
    create_dino_dynamo_table(**dynamodb_args)

Table 'dino_info' exists. Table status: ACTIVE


### Query The DynamoDB Table to retrieve City and DinoID given Date

In [17]:
# Example usage
date_to_query = '3/19/2024'
items = query_dynamodb_dino_tbl_by_date(date_to_query)
items

[{'DinoID_Transported': 'T88',
  'Date': '3/19/2024',
  '\ufeffRoute_Number': '123999',
  'City': 'Anchorage'}]

## SQL Agent

In [18]:
from langchain_community.utilities import SQLDatabase
from langchain_openai import OpenAI, ChatOpenAI
import openai

In [19]:
db = SQLDatabase.from_uri("sqlite:///../sqllite_dino_db/dino_name.db")
db.run("SELECT * FROM dinosaurs LIMIT 10;")

"[('T88', 'T-Rex'), ('V66', 'Velociraptor')]"

In [21]:
import sql_agent.sql_agent
importlib.reload(sql_agent.sql_agent)
from sql_agent.sql_agent import get_sql_agent

In [22]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
sqlagent = get_sql_agent(llm=llm, db_path = '../sqllite_dino_db/dino_name.db')

In [23]:
sqlagent.invoke(
    "What is the name of dinosaur with ID T88"
)['output']



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


dinosaurs
Invoking: `sql_db_schema` with `{'table_names': 'dinosaurs'}`



CREATE TABLE dinosaurs (
	"ID" TEXT, 
	"Name" TEXT, 
	PRIMARY KEY ("ID")
)

/*
3 rows from dinosaurs table:
ID	Name
T88	T-Rex
V66	Velociraptor
*/
Invoking: `sql_db_query` with `{'query': "SELECT Name FROM dinosaurs WHERE ID = 'T88'"}`


[('T-Rex',)]The dinosaur with ID T88 is named T-Rex.

> Finished chain.


'The dinosaur with ID T88 is named T-Rex.'

### Temperature

In [25]:
import react_agent
importlib.reload(react_agent)
from react_agent import get_react_agent, OpenAI_agent, get_all_tools


api_keys = {'OPENWEATHERMAP_API_KEY':os.getenv("OPENWEATHERMAP_API_KEY")}

In [26]:
tools = get_all_tools(load_api_keys=api_keys)
agent_executor = OpenAI_agent(model="gpt-3.5-turbo",
             tools=tools, 
             prompt=None, 
             hub_prompt="hwchase17/react", 
             temperature=0)


In [27]:
input_msg = 'What is the temperature in farenheight in Anchorage?'
agent_executor.invoke({"input": f"{input_msg}. Only use a tool if needed, otherwise respond with Final Answer"})



> Entering new AgentExecutor chain...
Anchorage is a location, so I should use the open_weather_map tool to fetch the current temperature.
Action: open_weather_map
Action Input: AnchorageIn Anchorage, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 8.75 m/s, direction: 160°
Humidity: 52%
Temperature: 
  - Current: 9.42°C
  - High: 11.56°C
  - Low: 6.59°C
  - Feels like: 5.73°C
Rain: {}
Heat index: None
Cloud cover: 100%I need to convert the current temperature in Celsius to Fahrenheit.
Action: celsius_to_fahrenheit_conversion
Action Input: 9.42°C48.956I now have the temperature in Fahrenheit for Anchorage.
Final Answer: The temperature in Fahrenheit in Anchorage is 48.956°F.

> Finished chain.


{'input': 'What is the temperature in farenheight in Anchorage?. Only use a tool if needed, otherwise respond with Final Answer',
 'output': 'The temperature in Fahrenheit in Anchorage is 48.956°F.'}

## End to End Testing

In [69]:
import sql_agent.sql_agent
importlib.reload(sql_agent.sql_agent)
from sql_agent.sql_agent import get_sql_agent

import react_agent
importlib.reload(react_agent)
from react_agent import get_react_agent, OpenAI_agent, get_all_tools

In [70]:
tools = get_all_tools(load_api_keys=api_keys)
agent_executor = OpenAI_agent(model="gpt-3.5-turbo",
             tools=tools, 
             prompt=None, 
             hub_prompt="hwchase17/react", 
             temperature=0)



In [17]:
input_msg = """What is the safe temperature range of the dinosaur being transported on date 3/19/2024? In your response,
               also give the city and name of the dinosaur as well."""
temperature_response = agent_executor.invoke({"input": f"{input_msg}. Only use a tool if needed, otherwise respond with Final Answer"})



> Entering new AgentExecutor chain...
I need to query the DynamoDB table to get the relevant information about the dinosaur being transported on 3/19/2024.
Action: query_dynamodb_dino_tbl_by_date
Action Input: 3/19/2024[{'DinoID_Transported': 'T88', 'Date': '3/19/2024', '\ufeffRoute_Number': '123999', 'City': 'Anchorage'}]Now that I have the DinoID_Transported and City information, I need to find out the dinosaur's name.
Action: query_dino_name_tool
Action Input: T88

> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


dinosaurs
Invoking: `sql_db_schema` with `{'table_names': 'dinosaurs'}`



CREATE TABLE dinosaurs (
	"ID" TEXT, 
	"Name" TEXT, 
	PRIMARY KEY ("ID")
)

/*
3 rows from dinosaurs table:
ID	Name
T88	T-Rex
V66	Velociraptor
*/
Invoking: `sql_db_query` with `{'query': "SELECT Name FROM dinosaurs WHERE ID = 'T88'"}`


[('T-Rex',)]The dinosaur with ID T88 is called T-Rex.

> Finished chain.
The dinosaur with ID T88 is called T-Rex.Now that I h

In [18]:
temperature_response['output']

'The safe temperature range for transporting the T-Rex on 3/19/2024 in Anchorage is 57°F-68°F.'

In [26]:
input_msg = f"""Given the following context: 
                {temperature_response['output']}
                what is the current temperature in fahrenheight of the referenced city? Give the answer like this example
                'The current temperature in CITY is TEMPERATURE"""
current_temp = agent_executor.invoke({"input": f"{input_msg}. Only use a tool if needed, otherwise respond with Final Answer"})



> Entering new AgentExecutor chain...
I need to find the current temperature in Fahrenheit for Anchorage on 3/19/2024 to determine if it falls within the safe range for transporting the T-Rex.
Action: open_weather_map
Action Input: Anchorage,USIn Anchorage,US, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 2.06 m/s, direction: 0°
Humidity: 62%
Temperature: 
  - Current: 8.64°C
  - High: 10.92°C
  - Low: 7.14°C
  - Feels like: 7.56°C
Rain: {}
Heat index: None
Cloud cover: 100%I need to convert the current temperature of 8.64°C to Fahrenheit.
Action: celsius_to_fahrenheit_conversion
Action Input: 8.6447.552The current temperature in Anchorage on 3/19/2024 is 47.552°F, which falls within the safe range for transporting the T-Rex.
Final Answer: The current temperature in Anchorage is 47.552°F.

> Finished chain.


In [71]:
temperature_response['output'] + ' ' + current_temp['output']

'The safe temperature range for transporting the T-Rex on 3/19/2024 in Anchorage is 57°F-68°F. The current temperature in Anchorage is 47.552°F.'

In [104]:
input_msg = f"""{temperature_response['output'] + ' ' + current_temp['output']}
                Is it safe to transport the dinosaur at that temperature? You do not need to do any temperature conversion."""
safety_response = agent_executor.invoke({"input": f"{input_msg}. Only use a tool if needed, otherwise respond with Final Answer"})



> Entering new AgentExecutor chain...
Since I already have the current temperature and the safe temperature range, I can directly use the dino_temp_safety_check tool to determine if it's safe to transport the dinosaur.
Action: dino_temp_safety_check
Action Input: ["47.552", "57", "68"]It is not safe for the dinosaurs at this temperature because it is too cold.I now know the final answer
Final Answer: It is not safe to transport the T-Rex at the current temperature in Anchorage because it is too cold.

> Finished chain.


In [110]:
temperature_response['output'] + ' ' + current_temp['output'] + ' ' + safety_response['output']

'The safe temperature range for transporting the T-Rex on 3/19/2024 in Anchorage is 57°F-68°F. The current temperature in Anchorage is 47.552°F. It is not safe to transport the T-Rex at the current temperature in Anchorage because it is too cold.'

In [114]:
input_msg = f"""Given this information: {temperature_response['output'] + ' ' + current_temp['output'] + ' ' + safety_response['output']}
                If it is not safe to transport the dinosaur, check what specific actions 
                we can take to to keep the relevant dinosaur safe based on the existing documentation (Make sure to reference the correct document since you are given the dinosaur name)"""
action_response = agent_executor.invoke({"input": f"{input_msg}. Only use a tool if needed, otherwise respond with Final Answer"})



> Entering new AgentExecutor chain...
The question already provides the dinosaur name and the fact that it's not safe to transport it due to cold temperature. I need to find the specific actions to keep the T-Rex safe in cold weather from the dinosaur_temperature_transport_info.
Action: dinosaur_temperature_transport_info
Action Input: T-Rexlocation and the actual time the T-Rexs were moved and the time they were moved back into the freighter. 6.2.10 If the transporter is unreachable, all efforts must be made to keep reaching out to the transporter till the transporter responds to the alert.  7 QUALITY DOCUMENTS 7.1 Research Dinosaur Welfare Check Log       XXXXXX   8 REFERENCES N/A

Recording Temperatures using Planet and Responding to Planet Alerts- Rev#10 Downloaded or Printed documents should be deleted/shredded within 1 week. Page 1 of 3  JP-ZOO-INST-24: Recording Temperatures for Transport of T-Rexs   1. PURPOSE This Work Instruction describes the DINO Transporter’s responsibil

In [115]:
action_response

{'input': 'Given this information: The safe temperature range for transporting the T-Rex on 3/19/2024 in Anchorage is 57°F-68°F. The current temperature in Anchorage is 47.552°F. It is not safe to transport the T-Rex at the current temperature in Anchorage because it is too cold.\n                If it is not safe to transport the dinosaur, check what specific actions \n                we can take to to keep the relevant dinosaur safe based on the existing documentation (Make sure to reference the correct document since you are given the dinosaur name). Only use a tool if needed, otherwise respond with Final Answer',
 'output': 'The DINO Transporter should record the current temperature from the TYR55 sensor located in the cargo bay of the dinosaur freighter on the Research Dinosaur Welfare Check Log. The Associate Director, Operations and Strategy should be responsible for the action plan when temperatures in the freighter cannot be controlled. If the transporter is unreachable, all e

In [102]:
if safety_response['output'] == 'False':
    input_msg = f"""Given this information: {temperature_response['output'] + ' ' + current_temp['output']} It is not safe to transport the dinosaur because the current temperature is out of range. 
                Check what specific actions we can take to to keep the relevant dinosaur safe based on the existing documentation (Make sure to reference the correct document since you are given the dinosaur name)"""
    action_response = agent_executor.invoke({"input": f"{input_msg}. Only use a tool if needed, otherwise respond with Final Answer"})



> Entering new AgentExecutor chain...
I need to find the specific actions to take to keep the T-Rex safe during transportation based on the existing documentation.
Action: dinosaur_temperature_transport_info
Action Input: T-Rexlocation and the actual time the T-Rexs were moved and the time they were moved back into the freighter. 6.2.10 If the transporter is unreachable, all efforts must be made to keep reaching out to the transporter till the transporter responds to the alert.  7 QUALITY DOCUMENTS 7.1 Research Dinosaur Welfare Check Log       XXXXXX   8 REFERENCES N/A

Recording Temperatures using Planet and Responding to Planet Alerts- Rev#10 Downloaded or Printed documents should be deleted/shredded within 1 week. Page 1 of 3  JP-ZOO-INST-24: Recording Temperatures for Transport of T-Rexs   1. PURPOSE This Work Instruction describes the DINO Transporter’s responsibility of keeping the T-Rexs safe if the current temperature is out of the safe temperature range, which is 57°F-68°F f

### Craft a message with actions to take

In [108]:
message_info = temperature_response['output'] + ' ' + current_temp['output'] + ' ' + safety_response['output'] + ' ' + action_response['output']

In [109]:
message_info

'The safe temperature range for transporting the T-Rex on 3/19/2024 in Anchorage is 57°F-68°F. The current temperature in Anchorage is 47.552°F. It is not safe to transport the T-Rex at the current temperature in Anchorage because it is too cold. The specific actions to take to keep the T-Rex safe during transportation in cold temperatures are: 1) The DINO Transporter should record the temperatures from the TYR55 sensor located in the cargo bay of every dinosaur freighter on the Research Dinosaur Welfare Check Log. 2) If the temperature is out of the safe range (57°F-68°F for T-Rexs), the Associate Director, Operations and Strategy is responsible for the action plan when temperatures in the freighter cannot be controlled. 3) If the transporter is unreachable, all efforts must be made to keep reaching out to the transporter till the transporter responds to the alert. These actions are outlined in the document JP-ZOO-INST-24: Recording Temperatures for Transport of T-Rexs.'

In [116]:
input_msg = f"""Given this information: {message_info}
                craft a status email to the manager."""
action_response = agent_executor.invoke({"input": f"{input_msg}. Only use a tool if needed, otherwise respond with Final Answer"})



> Entering new AgentExecutor chain...
The information provided is comprehensive and does not require the use of any tools to answer the question. I can directly draft an email to the manager with the given information.
Final Answer: Subject: Urgent: T-Rex Transportation Temperature Alert

Dear Manager,

I am writing to inform you about a critical situation regarding the transportation of our T-Rex on 3/19/2024 in Anchorage. The current temperature in Anchorage is 47.552°F, which is below the safe temperature range for T-Rex transportation (57°F-68°F). 

As per our protocol outlined in document JP-ZOO-INST-24: Recording Temperatures for Transport of T-Rexs, it is not safe to transport the T-Rex at the current temperature because it is too cold. 

Here are the specific actions we are taking to ensure the safety of the T-Rex:

1) The DINO Transporter is recording the temperatures from the TYR55 sensor located in the cargo bay of every dinosaur freighter on the Research Dinosaur Welfare 

In [112]:
import sql_agent.sql_agent
importlib.reload(sql_agent.sql_agent)
from sql_agent.sql_agent import get_sql_agent

import react_agent
importlib.reload(react_agent)
from react_agent import get_react_agent, OpenAI_agent, get_all_tools, agent_check_safety_actions

In [113]:
agent_check_safety_actions(agent_executor=agent_executor,
                           context_info=temperature_response['output'] + ' ' + current_temp['output'] + ' ' + safety_response['output'])



> Entering new AgentExecutor chain...
I need to find the specific actions to take to keep the T-Rex safe during transportation in cold weather. I can use the dinosaur_temperature_transport_info tool to get this information.
Action: dinosaur_temperature_transport_info
Action Input: T-Rexlocation and the actual time the T-Rexs were moved and the time they were moved back into the freighter. 6.2.10 If the transporter is unreachable, all efforts must be made to keep reaching out to the transporter till the transporter responds to the alert.  7 QUALITY DOCUMENTS 7.1 Research Dinosaur Welfare Check Log       XXXXXX   8 REFERENCES N/A

Recording Temperatures using Planet and Responding to Planet Alerts- Rev#10 Downloaded or Printed documents should be deleted/shredded within 1 week. Page 1 of 3  JP-ZOO-INST-24: Recording Temperatures for Transport of T-Rexs   1. PURPOSE This Work Instruction describes the DINO Transporter’s responsibility of keeping the T-Rexs safe if the current temperatur

'The DINO Transporter is responsible for recording temperatures from the TYR55 sensor located in the cargo bay of every dinosaur freighter on the Research Dinosaur Welfare Check Log. T-Rexs are not safe when temperatures are out of range (57°F-68°F). The Associate Director, Operations and Strategy is responsible for the action plan when temperatures in the freighter cannot be controlled. If the transporter is unreachable, all efforts must be made to keep reaching out to the transporter till the transporter responds to the alert.'

## Boto3 SES Message

In [7]:
import aws_features.aws_utils
importlib.reload(aws_features.aws_utils)
from aws_features.aws_utils import aws_send_sms, send_email

In [10]:
response = send_email(from_email='yongpeng@usc.edu',
                      to_email='billyptang@gmail.com',
                      subject='Dinosaur Safety',
                      body='Something')
print(response)

{'MessageId': '0100018f40a5afed-0cb6e07a-761d-4724-b5f9-63547e9da339-000000', 'ResponseMetadata': {'RequestId': 'dc38112b-7237-41c9-bc80-2c6e2c1cd611', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 03 May 2024 22:49:11 GMT', 'content-type': 'text/xml', 'content-length': '326', 'connection': 'keep-alive', 'x-amzn-requestid': 'dc38112b-7237-41c9-bc80-2c6e2c1cd611'}, 'RetryAttempts': 0}}


In [40]:
if response.get('MessageId') and response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Message sent successfully!")
else:
    print("Failed to send message.")

Message sent successfully!


# Part 4

In [122]:
import dino_safety_flow_part4
importlib.reload(dino_safety_flow_part4)
from dino_safety_flow_part4 import dino_temp_safety_check_workflow

In [123]:
email = dino_temp_safety_check_workflow(model_name='gpt-4',
                                        temperature=0,
                                        date='3/19/2024',
                                        phonenumber='+16319353837',
                                        api_keys=api_keys)



> Entering new AgentExecutor chain...
I need to find out the dinosaur being transported on the given date, its safe temperature range, the city it's being transported in, and its name. I will start by querying the DynamoDB table by date to get the Dino ID and City name.
Action: query_dynamodb_dino_tbl_by_date
Action Input: 3/19/2024[{'DinoID_Transported': 'T88', 'Date': '3/19/2024', '\ufeffRoute_Number': '123999', 'City': 'Anchorage'}]I now have the Dino ID (T88) and the city (Anchorage). Next, I need to find out the name of the dinosaur using the Dino ID.
Action: query_dino_name_tool
Action Input: T88

> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


dinosaurs
Invoking: `sql_db_schema` with `{'table_names': 'dinosaurs'}`



CREATE TABLE dinosaurs (
	"ID" TEXT, 
	"Name" TEXT, 
	PRIMARY KEY ("ID")
)

/*
3 rows from dinosaurs table:
ID	Name
T88	T-Rex
V66	Velociraptor
*/
Invoking: `sql_db_query` with `{'query': "SELECT Name FROM dinosaurs WHERE ID =

NameError: name 'sms_responsee' is not defined

# Other

In [31]:
import streamlit_chatbot
importlib.reload(streamlit_chatbot)
from streamlit_chatbot import initialize_data_sources

In [32]:
initialize_data_sources(chromadb_args={'doc_dir': '../pdf_data/', 
                                           'db_path': '../chroma_vector_store/'}, 
                            dynamodb_args={'table_name': 'dino_info',
                                           'csv_file_path': '../csv_data/dino_records.csv'}) 

Chroma DB already exists.
Table 'dino_info' exists. Table status: ACTIVE
